In [1]:
%load_ext autoreload
%autoreload 2

# Set up

In [2]:
# import necessery packages 
import os 
import numpy as np 
from pathlib import Path
import itertools
import pandas as pd

### Set up parameters

In [3]:
# all the finetuning parameters
param_grid = {
    'n_estimators': [10, 20, 30],
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': [None, 'sqrt'],
    'bootstrap': [True], 
}

# # all the finetuning parameters
# param_grid = {
#     'n_estimators': [1],
#     'criterion': ['gini'],
#     'max_depth': [None],
#     'min_samples_split': [2],
#     'min_samples_leaf': [1],
#     'max_features': [None],
#     'bootstrap': [True, False]
# }

# Tạo tất cả các tổ hợp tham số
param_combinations = list(itertools.product(
    param_grid['n_estimators'],
    param_grid['max_depth'],
    param_grid['criterion'],
    param_grid['min_samples_split'],
    param_grid['min_samples_leaf'],
    param_grid['max_features'],
    param_grid['bootstrap'],
))

### Set up data

In [4]:
# Get data directories
data_folder = '../data'
train_dir = os.path.join(data_folder, 'train.csv')
test_dir = os.path.join(data_folder, 'test.csv')
val_dir = os.path.join(data_folder, 'val.csv')

In [5]:
# Load data
def load_data(data_dir): 
    df = pd.read_csv(data_dir)
    X = df.drop(columns=['TARGET']) 
    y = df['TARGET'] 
    return X, y

X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)
X_val, y_val = load_data(val_dir)

### Set up folder for logging

In [6]:
experiment_dir = Path('./output/RandomForest')
experiment_dir.mkdir(parents=True, exist_ok=True)

# Random Forest from scratch

In [16]:
# Set up system path - dont care
import sys
project_root = Path('.').resolve().parent  # Lùi một cấp ra khỏi notebook/
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

import importlib
import train.random_forest_trainer
import model.random_forest
importlib.reload(model.random_forest)            # reload module phụ thuộc trước
importlib.reload(train.random_forest_trainer)   # reload module chính sau
from model.random_forest import RandomForest
from train.random_forest_trainer import RandomForestTrainer

In [17]:
# Lưu kết quả vào list rồi tạo DataFrame
results = []

for i, (n_estimators, max_depth, criterion, min_samples_split, min_samples_leaf, max_features, bootstrap) in enumerate(param_combinations):
    params = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'criterion': criterion,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'max_features': max_features,
        'bootstrap': bootstrap,
    }

    config = {'params': params}

    # Train
    trainer = RandomForestTrainer(config)
    model, train_metrics, val_metrics = trainer.train(X_train, y_train, X_val, y_val, experiment_dir)

    # Ghi nhận kết quả
    results.append({
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'criterion': criterion,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'max_features': max_features,
        'bootstrap': bootstrap,
        'accuracy': val_metrics['accuracy'],
        'precision': val_metrics['precision'],
        'recall': val_metrics['recall'],
        'f1': val_metrics['f1']
    })

# Chuyển sang DataFrame
scratch_results_df = pd.DataFrame(results)

# Lưu ra file CSV
scratch_results_df.to_csv(experiment_dir / 'scratch_finetune_results.csv', index=False)

# (Tùy chọn) In top cấu hình tốt nhất theo F1-score
print(scratch_results_df.sort_values(by='f1', ascending=False).head())

2025-06-03 20:41:29,229 - INFO - Training Random Forest from scratch with parameters: {'n_estimators': 10, 'max_depth': 10, 'criterion': 'gini', 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': True}


RandomForestScratch initialized with parameters:
  n_estimators     = 10
  criterion        = gini
  max_depth        = 10
  min_samples_split = 2
  min_samples_leaf  = 1
  max_features      = None
  bootstrap         = True
  max_samples       = None
  random_state      = 0
  max_leaf_nodes    = None


2025-06-03 20:42:54,758 - INFO - Training metrics:
Accuracy: 0.7729
Precision: 0.7794
Recall: 0.7729
F1 Score: 0.7755
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.82      0.83     92140
           1       0.62      0.67      0.64     40529

    accuracy                           0.77    132669
   macro avg       0.73      0.75      0.74    132669
weighted avg       0.78      0.77      0.78    132669



2025-06-03 20:42:54,759 - INFO - Validation results:
Accuracy: 0.7658
Precision: 0.7616
Recall: 0.7658
F1 Score: 0.7631
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.85      0.83     29487
           1       0.66      0.61      0.63     14736

    accuracy                           0.77     44223
   macro avg       0.74      0.73      0.73     44223
weighted avg       0.76      0.77      0.76     44223

2025-06-03 20:42:55,997 - INFO - Training Random Forest from

RandomForestScratch initialized with parameters:
  n_estimators     = 10
  criterion        = gini
  max_depth        = 10
  min_samples_split = 2
  min_samples_leaf  = 1
  max_features      = sqrt
  bootstrap         = True
  max_samples       = None
  random_state      = 0
  max_leaf_nodes    = None


d:\Anaconda\envs\mlproject\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Anaconda\envs\mlproject\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Anaconda\envs\mlproject\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Anaconda\envs\mlproject\Lib\site-packages\sklearn\metri

...

RandomForestScratch initialized with parameters:
  n_estimators     = 30
  criterion        = entropy
  max_depth        = 20
  min_samples_split = 2
  min_samples_leaf  = 1
  max_features      = None
  bootstrap         = True
  max_samples       = None
  random_state      = 0
  max_leaf_nodes    = None


2025-06-03 23:14:53,693 - INFO - Training metrics:
Accuracy: 0.8282
Precision: 0.8330
Recall: 0.8282
F1 Score: 0.8300
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.86      0.87     91841
           1       0.70      0.76      0.73     40828

    accuracy                           0.83    132669
   macro avg       0.80      0.81      0.80    132669
weighted avg       0.83      0.83      0.83    132669



2025-06-03 23:14:53,773 - INFO - Validation results:
Accuracy: 0.7486
Precision: 0.7439
Recall: 0.7486
F1 Score: 0.7456
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.83      0.82     29487
           1       0.63      0.58      0.61     14736

    accuracy                           0.75     44223
   macro avg       0.72      0.71      0.71     44223
weighted avg       0.74      0.75      0.75     44223

2025-06-03 23:21:03,057 - INFO - Training Random Forest from

RandomForestScratch initialized with parameters:
  n_estimators     = 30
  criterion        = entropy
  max_depth        = 20
  min_samples_split = 2
  min_samples_leaf  = 1
  max_features      = sqrt
  bootstrap         = True
  max_samples       = None
  random_state      = 0
  max_leaf_nodes    = None


2025-06-03 23:26:34,156 - INFO - Training metrics:
Accuracy: 0.6795
Precision: 0.9787
Recall: 0.6795
F1 Score: 0.7925
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.68      0.81    130262
           1       0.05      0.85      0.09      2407

    accuracy                           0.68    132669
   macro avg       0.52      0.76      0.45    132669
weighted avg       0.98      0.68      0.79    132669



2025-06-03 23:26:34,167 - INFO - Validation results:
Accuracy: 0.6801
Precision: 0.7372
Recall: 0.6801
F1 Score: 0.5677
Classification Report:
              precision    recall  f1-score   support

           0       0.68      1.00      0.81     29487
           1       0.86      0.05      0.09     14736

    accuracy                           0.68     44223
   macro avg       0.77      0.52      0.45     44223
weighted avg       0.74      0.68      0.57     44223



    n_estimators  max_depth criterion  min_samples_split  min_samples_leaf  \
14            20         10   entropy                  2                 1   
24            30         10      gini                  2                 1   
26            30         10   entropy                  2                 1   
0             10         10      gini                  2                 1   
2             10         10   entropy                  2                 1   

   max_features  bootstrap  accuracy  precision    recall        f1  
14         None       True  0.768582   0.762816  0.768582  0.764050  
24         None       True  0.767383   0.761925  0.767383  0.763302  
26         None       True  0.767881   0.762013  0.767881  0.763236  
0          None       True  0.765801   0.761613  0.765801  0.763062  
2          None       True  0.766140   0.761544  0.766140  0.763020  


In [18]:
# get best hyperparameter set
best_result = scratch_results_df.loc[scratch_results_df['f1'].idxmax()]
best_params = {
    'n_estimators': best_result['n_estimators'],
    'max_depth': best_result['max_depth'],
    'criterion': best_result['criterion'],
    'min_samples_split': best_result['min_samples_split'],
    'min_samples_leaf': best_result['min_samples_leaf'],
    'max_features': best_result['max_features'],
    'bootstrap': best_result['bootstrap'],
}

print("Best parameters based on F1-score:")
print(best_params)
print("\nValidation metrics:")
print(f"Accuracy: {best_result['accuracy']:.4f}")
print(f"Precision: {best_result['precision']:.4f}")
print(f"Recall: {best_result['recall']:.4f}")
print(f"F1-score: {best_result['f1']:.4f}")


Best parameters based on F1-score:
{'n_estimators': np.int64(20), 'max_depth': np.int64(10), 'criterion': 'entropy', 'min_samples_split': np.int64(2), 'min_samples_leaf': np.int64(1), 'max_features': None, 'bootstrap': np.True_}

Validation metrics:
Accuracy: 0.7686
Precision: 0.7628
Recall: 0.7686
F1-score: 0.7640


In [10]:
# plot into chart 

# Random Forest Sklearn

In [11]:
# import lib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import logging

In [12]:
len(param_combinations)

36

In [13]:
results = []

for i, (n_estimators, max_depth, criterion, min_samples_split,
        min_samples_leaf, max_features, bootstrap) in enumerate(param_combinations):
    print(i)
    params = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'criterion': criterion,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'max_features': max_features,
        'bootstrap': bootstrap,
        'random_state': 42,
        'n_jobs': -1
    }
   # Log cấu hình model
    logging.info(f"Training Random Forest using sklearn with parameters: {params}")

    # Khởi tạo model sklearn
    model = RandomForestClassifier(**params)

    # Train model
    model.fit(X_train, y_train)

    # Dự đoán
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)

    # Tính toán metrics
    train_metrics = {
        'accuracy': accuracy_score(y_train, y_pred_train),
        'precision': precision_score(y_train, y_pred_train, average='macro'),
        'recall': recall_score(y_train, y_pred_train, average='macro'),
        'f1': f1_score(y_train, y_pred_train, average='macro')
    }

    val_metrics = {
        'accuracy': accuracy_score(y_val, y_pred_val),
        'precision': precision_score(y_val, y_pred_val, average='macro'),
        'recall': recall_score(y_val, y_pred_val, average='macro'),
        'f1': f1_score(y_val, y_pred_val, average='macro')
    }

    # Lưu kết quả
    results.append({
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'criterion': criterion,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'max_features': max_features,
        'bootstrap': bootstrap,
        'accuracy': val_metrics['accuracy'],
        'precision': val_metrics['precision'],
        'recall': val_metrics['recall'],
        'f1': val_metrics['f1']
    })

    # Log training metrics
    logging.info("Training metrics:")
    logging.info(f"Accuracy: {train_metrics['accuracy']:.4f}")
    logging.info(f"Precision: {train_metrics['precision']:.4f}")
    logging.info(f"Recall: {train_metrics['recall']:.4f}")
    logging.info(f"F1 Score: {train_metrics['f1']:.4f}")
    logging.info("Classification Report:\n" + classification_report(y_train, y_pred_train))

    # Log validation metrics
    logging.info("Validation results:")
    logging.info(f"Accuracy: {val_metrics['accuracy']:.4f}")
    logging.info(f"Precision: {val_metrics['precision']:.4f}")
    logging.info(f"Recall: {val_metrics['recall']:.4f}")
    logging.info(f"F1 Score: {val_metrics['f1']:.4f}")
    logging.info("Classification Report:\n" + classification_report(y_val, y_pred_val))

# Chuyển thành DataFrame
sklearn_results_df = pd.DataFrame(results)

# Lưu kết quả vào CSV
sklearn_results_df.to_csv(experiment_dir / 'sklearn_finetune_results.csv', index=False)

# In ra top 5 theo f1-score
print(sklearn_results_df.sort_values(by='f1', ascending=False).head())

2025-06-03 18:24:21,521 - INFO - Training Random Forest using sklearn with parameters: {'n_estimators': 10, 'max_depth': 10, 'criterion': 'gini', 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': True, 'random_state': 42, 'n_jobs': -1}


0


2025-06-03 18:24:22,383 - INFO - Training metrics:
2025-06-03 18:24:22,384 - INFO - Accuracy: 0.7766
2025-06-03 18:24:22,385 - INFO - Precision: 0.7508
2025-06-03 18:24:22,386 - INFO - Recall: 0.7325
2025-06-03 18:24:22,387 - INFO - F1 Score: 0.7397
2025-06-03 18:24:22,426 - INFO - Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.86      0.84     88462
           1       0.69      0.60      0.64     44207

    accuracy                           0.78    132669
   macro avg       0.75      0.73      0.74    132669
weighted avg       0.77      0.78      0.77    132669

2025-06-03 18:24:22,427 - INFO - Validation results:
2025-06-03 18:24:22,428 - INFO - Accuracy: 0.7657
2025-06-03 18:24:22,429 - INFO - Precision: 0.7376
2025-06-03 18:24:22,430 - INFO - Recall: 0.7199
2025-06-03 18:24:22,431 - INFO - F1 Score: 0.7267
2025-06-03 18:24:22,443 - INFO - Classification Report:
              precision    recall  f1-score   support

    

1


2025-06-03 18:24:22,920 - INFO - Training metrics:
2025-06-03 18:24:22,920 - INFO - Accuracy: 0.7707
2025-06-03 18:24:22,922 - INFO - Precision: 0.7500
2025-06-03 18:24:22,925 - INFO - Recall: 0.7103
2025-06-03 18:24:22,927 - INFO - F1 Score: 0.7221
2025-06-03 18:24:22,960 - INFO - Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.89      0.84     88462
           1       0.71      0.53      0.61     44207

    accuracy                           0.77    132669
   macro avg       0.75      0.71      0.72    132669
weighted avg       0.76      0.77      0.76    132669

2025-06-03 18:24:22,961 - INFO - Validation results:
2025-06-03 18:24:22,962 - INFO - Accuracy: 0.7660
2025-06-03 18:24:22,965 - INFO - Precision: 0.7438
2025-06-03 18:24:22,967 - INFO - Recall: 0.7050
2025-06-03 18:24:22,967 - INFO - F1 Score: 0.7164
2025-06-03 18:24:22,979 - INFO - Classification Report:
              precision    recall  f1-score   support

    

...

34


2025-06-03 18:25:08,684 - INFO - Training metrics:
2025-06-03 18:25:08,685 - INFO - Accuracy: 0.8315
2025-06-03 18:25:08,686 - INFO - Precision: 0.8120
2025-06-03 18:25:08,687 - INFO - Recall: 0.8049
2025-06-03 18:25:08,688 - INFO - F1 Score: 0.8083
2025-06-03 18:25:08,723 - INFO - Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.88      0.88     88462
           1       0.76      0.73      0.74     44207

    accuracy                           0.83    132669
   macro avg       0.81      0.80      0.81    132669
weighted avg       0.83      0.83      0.83    132669

2025-06-03 18:25:08,724 - INFO - Validation results:
2025-06-03 18:25:08,725 - INFO - Accuracy: 0.7441
2025-06-03 18:25:08,726 - INFO - Precision: 0.7113
2025-06-03 18:25:08,727 - INFO - Recall: 0.7052
2025-06-03 18:25:08,728 - INFO - F1 Score: 0.7079
2025-06-03 18:25:08,743 - INFO - Classification Report:
              precision    recall  f1-score   support

    

35


2025-06-03 18:25:10,306 - INFO - Training metrics:
2025-06-03 18:25:10,307 - INFO - Accuracy: 0.8304
2025-06-03 18:25:10,307 - INFO - Precision: 0.8109
2025-06-03 18:25:10,309 - INFO - Recall: 0.8031
2025-06-03 18:25:10,310 - INFO - F1 Score: 0.8067
2025-06-03 18:25:10,350 - INFO - Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.88      0.87     88462
           1       0.76      0.72      0.74     44207

    accuracy                           0.83    132669
   macro avg       0.81      0.80      0.81    132669
weighted avg       0.83      0.83      0.83    132669

2025-06-03 18:25:10,352 - INFO - Validation results:
2025-06-03 18:25:10,353 - INFO - Accuracy: 0.7452
2025-06-03 18:25:10,354 - INFO - Precision: 0.7126
2025-06-03 18:25:10,355 - INFO - Recall: 0.7057
2025-06-03 18:25:10,357 - INFO - F1 Score: 0.7088
2025-06-03 18:25:10,381 - INFO - Classification Report:
              precision    recall  f1-score   support

    

    n_estimators  max_depth criterion  min_samples_split  min_samples_leaf  \
24            30         10      gini                  2                 1   
14            20         10   entropy                  2                 1   
26            30         10   entropy                  2                 1   
12            20         10      gini                  2                 1   
0             10         10      gini                  2                 1   

   max_features  bootstrap  accuracy  precision    recall        f1  
24         None       True  0.767203   0.739617  0.720821  0.728037  
14         None       True  0.767971   0.740951  0.720176  0.727960  
26         None       True  0.768265   0.741554  0.719666  0.727755  
12         None       True  0.766750   0.739009  0.720550  0.727662  
0          None       True  0.765665   0.737565  0.719872  0.726744  


In [14]:
# get best hyperparameter set
best_result = sklearn_results_df.loc[sklearn_results_df['f1'].idxmax()]
best_params = {
    'n_estimators': best_result['n_estimators'],
    'max_depth': best_result['max_depth'],
    'criterion': best_result['criterion'],
    'min_samples_split': best_result['min_samples_split'],
    'min_samples_leaf': best_result['min_samples_leaf'],
    'max_features': best_result['max_features'],
    'bootstrap': best_result['bootstrap'],
}

print("Best parameters based on F1-score:")
print(best_params)
print("\nValidation metrics:")
print(f"Accuracy: {best_result['accuracy']:.4f}")
print(f"Precision: {best_result['precision']:.4f}")
print(f"Recall: {best_result['recall']:.4f}")
print(f"F1-score: {best_result['f1']:.4f}")


Best parameters based on F1-score:
{'n_estimators': np.int64(30), 'max_depth': np.int64(10), 'criterion': 'gini', 'min_samples_split': np.int64(2), 'min_samples_leaf': np.int64(1), 'max_features': None, 'bootstrap': np.True_}

Validation metrics:
Accuracy: 0.7672
Precision: 0.7396
Recall: 0.7208
F1-score: 0.7280


In [15]:
# plot into charts using finetune_results.csv